In [55]:
!git clone https://github.com/eujhwang/meme-cap.git

Cloning into 'meme-cap'...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 52 (delta 10), reused 13 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (52/52), 844.03 KiB | 11.25 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [1]:
import datasets
dataset = datasets.load_dataset('json', data_files=['./meme-cap/data/memes-trainval.json','./meme-cap/data/memes-test.json'], split="train")

/home/ahmed.attia/.conda/envs/nlp-meme/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Install Libraries

In [2]:
# !pip install timm
# !pip install flash-attn==2.0.6
# !pip install transformers
# !pip install accelerate 
# !pip install einops 

Import Florence-2 Model

In [3]:
# !mkdir my_models
# !mkdir my_models/Florence_2

In [4]:
# !pip install opencv-python

In [2]:
import torch
torch.cuda.empty_cache()

In [2]:
from transformers import AutoModelForCausalLM, AutoProcessor
model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large-ft",
                                             cache_dir="./my_models/Florence_2",
                                             device_map="cuda",
                                             trust_remote_code=True)

processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large-ft", trust_remote_code=True)

/home/ahmed.attia/.conda/envs/nlp-meme/lib/python3.9/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Florence2LanguageForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `Pr

In [3]:
import requests
from PIL import Image
import os
import cv2
from PIL import Image
import numpy as np

def download_image(url, file_path):
  """Downloads an image from a URL and saves it to a file.

  Args:
    url: The URL of the image.
    file_path: The path to save the image to.
  """
  try:
    response = requests.get(url, stream=True)
    response.raise_for_status()  # Raise an exception for bad status codes

    with open(file_path, 'wb') as file:
      for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)

    print(f"Image downloaded and saved to: {file_path}")
    return True
  except requests.exceptions.RequestException as e:
    print(f"Error downloading image: {e}")
    return False

def ocr_image(batch):
  prompt = "<OCR_WITH_REGION>"
  input_images = [process_image_from_url(url) for url in batch["url"]]
  # remove Nones 
  cleaned_list = [x for x in input_images if x is not None]
  if not len(cleaned_list):
    batch["extracted_text"] = [{'quad_boxes': [[None]],'labels': [None]} for _ in range(len(batch["url"]))]
    return batch
  inputs = processor(text=[prompt for _ in range(len(cleaned_list))], images=cleaned_list, return_tensors="pt").to("cuda")
  generated_ids = model.generate(
      input_ids=inputs["input_ids"],
      pixel_values=inputs["pixel_values"],
      max_new_tokens=1024,
      do_sample=False,
      num_beams=3)
  generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)
  predictions = ["" for _ in range(len(input_images))]
  pred_counter = 0 
  for i in range(len(input_images)):
    if input_images[i] == None:
      predictions[i] = {'<OCR_WITH_REGION>':{
    'quad_boxes': [[None]],
    'labels': [None ]}
}
    else:
      predictions[i] = processor.post_process_generation(generated_text[pred_counter], task="<OCR_WITH_REGION>", image_size=(inputs["pixel_values"][pred_counter].size()[1], inputs["pixel_values"][pred_counter].size()[0]))
      pred_counter+=1
  batch["extracted_text"] = predictions
  return batch


def preprocess_image(pil_image):
    # Convert the PIL image to a NumPy array
    img = np.array(pil_image.convert('L'))  # Convert to grayscale
    # Apply thresholding for better contrast
    _, img = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY)
    # Expand dimensions to make it 3-dimensional (H, W, C)
    img = np.expand_dims(img, axis=-1)  # Add a channel dimension
    img = np.concatenate([img, img, img], axis=-1)  # Duplicate the channel for RGB shape
    # Convert back to a PIL image
    processed_image = Image.fromarray(img)
    return processed_image

def process_image_from_url(image_url):
  """Downloads an image, preprocesses it, and extracts text using OCR."""
  file_name = "./imgs/"+image_url.split("/")[-1]
  # download pil image
  exist = download_image(image_url, file_name)
  if not exist:
    return None
  # Read the downloaded image
  image = Image.open(file_name)
  preprocessed_image = preprocess_image(image)
  return preprocessed_image

In [4]:
# split to save chunks
def split_dataset(dataset, n_chunks):
    total_size = len(dataset)
    chunk_size = total_size // n_chunks

    indices = np.array_split(range(total_size), n_chunks)
    return [dataset.select(idx) for idx in indices]

In [5]:
n_chunks = 60
chunks = split_dataset(dataset, n_chunks)
processed_chunks = []
for idx, chunk in enumerate(chunks):
    processed_chunk = chunk.map(ocr_image, batched=True, batch_size=3)
    processed_chunk.save_to_disk(f"chkpt/processed_chunk_{idx}")
    processed_chunks.append(processed_chunk)

Map:   0%|          | 0/107 [00:00<?, ? examples/s]

Image downloaded and saved to: ./imgs/ea217vacvgy91.jpg
Image downloaded and saved to: ./imgs/gmnwi9dxdfy91.jpg
Image downloaded and saved to: ./imgs/pv0v1dctvgy91.jpg


Map:   3%|▎         | 3/107 [00:06<03:38,  2.10s/ examples]

Image downloaded and saved to: ./imgs/tesq6y4nefy91.jpg
Image downloaded and saved to: ./imgs/7lfxftikwgy91.jpg
Image downloaded and saved to: ./imgs/apg5suwjffy91.jpg


Map:   6%|▌         | 6/107 [00:09<02:24,  1.43s/ examples]

Image downloaded and saved to: ./imgs/dot830boffy91.jpg
Image downloaded and saved to: ./imgs/63qwzae7xgy91.jpg
Image downloaded and saved to: ./imgs/p2qq4wt6gfy91.jpg


Map:   8%|▊         | 9/107 [00:13<02:17,  1.40s/ examples]

Image downloaded and saved to: ./imgs/ruogfvlzgfy91.jpg
Image downloaded and saved to: ./imgs/9wzhvw3hjfy91.jpg
Image downloaded and saved to: ./imgs/vq903azv1hy91.jpg


Map:  11%|█         | 12/107 [00:16<01:58,  1.24s/ examples]

Image downloaded and saved to: ./imgs/8t69jj61mfy91.jpg
Image downloaded and saved to: ./imgs/u18jokk5pfy91.jpg


In [28]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 2            |        cudaMalloc retries: 2         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  11513 MiB |  11569 MiB |  34454 MiB |  22940 MiB |
|       from large pool |  11491 MiB |  11547 MiB |  34330 MiB |  22838 MiB |
|       from small pool |     21 MiB |     22 MiB |    124 MiB |    102 MiB |
|---------------------------------------------------------------------------|
| Active memory         |  11513 MiB |  11569 MiB |  34454 MiB |  22940 MiB |
|       from large pool |  11491 MiB |  11547 MiB |  34330 MiB |

In [29]:
import torch
import gc
# Collect garbage
gc.collect()

# Clear cache
torch.cuda.empty_cache()


In [35]:
import torch
import torch.nn as nn
import torch.optim as optim

# Helper function to check GPU memory usage
def print_gpu_memory():
    print(f"Allocated memory: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")
    print(f"Cached memory: {torch.cuda.memory_reserved() / 1024**2:.2f} MB")

# Step 1: Create a model and optimizer, and allocate some tensors on the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = nn.Linear(1000, 1000).to(device)  # Model on GPU
optimizer = optim.Adam(model.parameters())  # Optimizer for the model
tensor = torch.randn(1000, 1000, device=device)  # Tensor on GPU

print("Before deleting references:")
print_gpu_memory()

# Step 2: Delete the references to these objects
del model
del optimizer
del tensor

print("\nAfter deleting references:")
print_gpu_memory()

# Step 3: Free up GPU memory manually using torch.cuda.empty_cache()
torch.cuda.empty_cache()

print("\nAfter emptying cache:")
print_gpu_memory()

Before deleting references:
Allocated memory: 11500.98 MB
Cached memory: 11598.00 MB

After deleting references:
Allocated memory: 11493.00 MB
Cached memory: 11598.00 MB

After emptying cache:
Allocated memory: 11493.00 MB
Cached memory: 11598.00 MB
